<span style="color:red">Abgegeben von (Name, Vorname):</span> 
**Elsherif, Mohamed**

In [1]:
import nltk

Immer griffbereit:
- Website: https://www.nltk.org/
- Buch: https://www.nltk.org/book/ch07.html
- Module: https://www.nltk.org/py-modindex.html
- Beispiele: http://www.nltk.org/howto/


<p style="line-height:1.4"><font size="6"><strong>9. Sitzung: Named-Entity Recognition und featurebasierte Klassifizierer</strong></font></p>

In der letzten Sitzung wurden Verfahren behandelt, größere Entitäten in einem Text zu identifizieren – die Chunks. In dieser Sitzung geht es darum, diese Entitäten weiter semantisch zu klassifizieren. Dafür greifen wir auf eine neue Methode zurück: Feature-basierte Klassifizierer! 

# Information Extraction

Unter [Information Extraction (IE)](https://en.wikipedia.org/wiki/Information_extraction) versteht man die semantische Strukturierung unstruktierter Daten, d.h. die Erkennung von (zuvor festgelegten) **Entitätentypen** und **Relationen**, die zwischen diesen Entitäten bestehen.

In unserem Fall haben wir es mit natürlichsprachlichen Texten zu tun, die auf diese Weise strukturiert werden sollen. Ein Extraktionsziel könnten beispielsweise Organisationen und deren Standort sein, wie im folgenden Satz aus dem NLTK-Buch:

> **<span style="color:green">BBDO South</span>** in **<span style="color:green">Atlanta</span>**, which handles corporate advertising for **<span style="color:green">Georgia-Pacific</span>**, will assume additional duties for brands like Angel Soft toilet tissue and Sparkle paper towels, said Ken Haldin, a spokesman for **<span style="color:green">Georgia-Pacific</span>** in **<span style="color:green">Atlanta</span>**.

Die extrahierte Information entspräche dann der folgenden Tabelle:

| **Organization** | **Location** |
| :--------------- | :----------- |
| BBDO South       | Atlanta      |
| Georgia-Pacific  | Atlanta      |

Der Unterschied ist, dass `Organization` die Eigenschaft einer Entität ist, `Location` aber die Relation zwischen *zwei* Entitäten: *Atlanta* ist die `Location` von *BBDO South*.    

Es lassen sich also zwei NLP-Aufgaben unterscheiden, mit denen wir uns in diesem und dem nächsten Notebook beschäftigen werden:

1. **Entitätenextraktion (Entity Extraction):** die Identifikation und Kategorisierung von Entitäten 
2. **Relationsextraktion (Relation Extraction):** die Identifikation und Kategorisierung von Relationen zwischen Entitäten 

Mit der Indentifikation von Entitäten, nämlich dem Chunking, haben wir uns bereits in der letzten Sitzung beschäftigt. In dieser Sitzung soll es um die semantische Grobklassifierung bestimmter Entitäten gehen, der sogenannten **Named Entities**.   

# Named-Entity Recognition (NER)

**Named Entities (NE)** sind durch einen **[Eigennamen](https://de.wikipedia.org/wiki/Eigenname)** bezeichnete einzelne Entitäten: 

- *President Obama* ist die Bezeichnung für ein bestimmte, einzelne Person
- *Tübingen* ist die Bezeichnung für eine bestimmte, einzelne Stadt 

Solche Bezeichnungen in einem Text gelten als NEs. 

**Achtung:** 
- *der erste afroamerikanische Präsident* 
- *die schönste Stadt am Neckar* 

sind keine NEs, obwohl offensichtlich eine einzelne Entität gemeint sein kann. *Präsident* und *Stadt* sind sogenannte **[Gattungsnamen](https://de.wikipedia.org/wiki/Gattungsname)**, aber keine Eigennamen. (Mit der Disambiguierung von Gattungsnamen, der Word-Sense Disambiguation (WSD), haben wir und schon beschäftigt.) 

Die Grenzen zwischen Eigennamen und Gattungsnamen sind aber nicht immer eindeutig. *Neckar* ist zwar ein Eigenname, aber (i) *Stadt am Neckar* ist wohl eher ein Gattungsname und (ii) *Neckar* außerdem nicht Teil des Chunks (weil Chunks nicht-rekursiv sind).   

Im NLTK-Buch werden die folgenden **NE-Typen** unterschieden.


| **NE Type**  | **Examples**                            |
| :----------- | :-------------------------------------- |
| ORGANIZATION | Georgia-Pacific Corp., WHO              |
| PERSON       | Eddy Bonte, President Obama             |
| LOCATION     | Murray River, Mount Everest             |
| DATE         | June, 2008-06-29                        |
| TIME         | two fifty a m, 1:30 p.m.                |
| MONEY        | 175 million Canadian Dollars, GBP 10.40 |
| PERCENT      | twenty pct, 18.75 %                     |
| FACILITY     | Washington Monument, Stonehenge         |
| GPE (Geopolitical Entity)          | South East Asia, Midlothian             |

Meist beschränkt man sich aber auf eine bestimmte Teilmenge wie ORGANIZATION, PERSON und LOCATION (siehe unten).

Die Extraktion von NEs, die sogenannte **Named-Entity Recognition (NER)**, beinhaltet dann die folgenden beiden Aufgaben:

1. den **Umfang** einer NE erkennen
2. den **Typ** einer NE erkennen

Die erste Teilaufgabe wird im Prinzip bereits vom Chunker erledigt. Doch wie können wir die zweite Aufgabe angehen? Wie schwer ist diese Aufgabe überhaupt?

## Herausforderungen der Tokenerkennung

Nach dem Chunking liegt eine Liste von NPs vor und die Herausforderung besteht zunächst darin, zu entscheiden, ob es sich dabei jeweils um eine NE handelt oder doch nur um einen Gattungsnamen. 

Sehen wir uns als Beispiel die Chunks aus der Pipline an:

    (S
     (NP BBDO/NNP South/NNP)
     (PP in/IN)
     (NP Atlanta/NNP)
     ,/,
     (NP which/WDT)
     (VP handles/VBZ)
     (NP corporate/JJ advertising/NN)
     (PP for/IN)
     (NP Georgia-Pacific/NNP)
      ,/,
     (VP will/MD assume/VB)
     (NP additional/JJ duties/NNS)
     (PP for/IN)
     (NP brands/NNS)
     (PP like/IN)
     (NP Angel/NNP Soft/NNP toilet/NN tissue/NN)
     and/CC
     (NP Sparkle/NNP paper/NN towels/NNS)
     ,/,
     (VP said/VBD)
     (NP Ken/NNP Haldin/NNP)
     ,/,
     (NP a/DT spokesman/NN)
     (PP for/IN)
     (NP Georgia-Pacific/NNP)
     (PP in/IN)
     (NP Atlanta/NNP))

Im Englischen könnte man die **Schreibung des Chunks** als Entscheidungsgrundlage verwenden: 

- Der Kopf (d.h. in der Regel das letzte Wort im Chunk) wird groß geschrieben. $\to$ Eine NE liegt vor.

Das funktioniert aber nicht immer: *Angel Soft toilet tissue* sieht nicht wie ein Eigenname aus, da *tissue* (der Kopf) klein geschrieben ist. Das scheint zunächst auch semantisch sinnvoll zu sein, da es sich um einen **Gattungssame mit einem integrierten Eigennamen** handeln könnte.

Im Beispielsatz heißt es aber: 

> [...] brands like Angel Soft toilet tissue and Sparkle paper towels [...]

D.h. *Angel Soft toilet tissue* ist eigentlich der Eigenname einer Marke. Die Schreibung der Chunks ist also nicht immer ein zuverlässiger Indikator. 

Daher besteht ein anderes Verfahren der Tokenerkennung darin, eine **Liste bekannter NEs** zu konsultieren. Die Erstellung und Pflege dieser Liste ist aber naturgemäß aufwendig und die Abdeckung trotzdem lückenhaft, da NE stark von der Domäne abhängen und einem ständigen Wandel unterliegen.  

Aus Mangel an Alternativen spielen diese NE-Listen aber bei der nachfolgenden Typerkennung eine wichtige Rolle.

## Herausforderungen der Typerkennung

Sind die NE-Token ermittelt, müssen sie einem Typ zugeordnet werden, also ORGANIZATION, PERSON, LOCATION oder MISC. 

Die Zuordnung kann per **Lookup in nach Typen getrennten NE-Listen** erfolgen, wobei sich wieder die Frage stellt, wie man diese NE-Listen erstellt und pflegt. 

Aber auch bei ausreichendem lexikalischem Wissen ist eine Typenzuordnung nicht immer eindeutig und die NP-Chunks diesbezüglich **ambig**. Im NLTK-Buch werden Beispiele genannt:

- *North* $\to$ LOCATION oder PERSON  
- *Christian Dior* $\to$ PERSON oder ORGANIZATON

Welcher NE-Typ jeweils intendiert ist, muss über den **Kontext** erschlossen werden.

Ein eher übertriebenes Beispiel für die Ambiguität bei der NER liefert das NLTK-Buch:

<img src="https://www.nltk.org/images/locations.png" alt="Drawing"/>

Hier wird die Orthographie, die POS-Tags und die Chunk-Struktur eben gerade nicht berücksichtigt.

Eine **Eigenheit komplexer NEs** ist außerdem, dass sie aus NEs von einem anderen Typ bestehen können. Das NLTK-Buch nennt hier die Beispiele:
- *Stanford University* (LOCATION/ORGANIZATION) 
- *Cecil H. Green Library* (PERSON/ORGANIZATION) 

Wenn diese komplexen Mehrwort-NEs unbekannt sein sollten, muss bei der Typenzuordnung die Struktur berücksichtigt werden, d.h. bei *Standford University* ist der Kopf *University* aber nicht der Modifizierer *Stanford* maßgeblich.   

# Methoden der NER

## CoNLL-2003 Shared Task: Vorbereitung der Trainings- und Testdaten

Im Rahmen der CoNLL-Konferenz wurden 2002 und 2003 Shared Tasks für die NER veranstaltet. Im NLTK sind die Daten aus dem Jahr 2002 enthalten (`from nltk.corpus import conll2002`). Leider waren die Zielsprachen dort Spanisch und Niederländisch. Wir werden daher im Folgenden die Daten aus dem Shared Task von 2003 nutzen.

Hier vorab die Berichte dieser beiden Shared Tasks:

- 2002: https://www.aclweb.org/anthology/W02-2024/
- 2003: https://www.aclweb.org/anthology/W03-0419/

### Einlesen der Daten

Die Daten des CoNLL-2003 Shared Task müssen nachinstalliert werden. Dafür benötigt man die drei Dateien in der beiligenden ZIP-Datei:

- `conll2003/eng.testa` (Development)
- `conll2003/eng.testb`
- `conll2003/eng.train`

Die Daten stammen übrigens aus dem [Reuters Corpus](https://trec.nist.gov/data/reuters/reuters.html), das auch für die Textklassifizierung benutzt wird und nicht frei verfügbar ist.

Glücklicherweise gibt es im NLTK einen "Reader" für das Einlesen von Corpora im CoNLL-Format. Die Methode `ConllCorpusReader` verlangt nur die Angabe des Verzeichnisses und der Spaltentypen, schon kann man mit `iob_words()`, `iob_sentences()` etc. das Corpus wie gewohnt auslesen:

In [2]:
from nltk.corpus import ConllCorpusReader
conll2003 = ConllCorpusReader(
    'conll2003/', 'eng.t.*', columntypes=('words', 'pos', 'chunk', 'ne'))

print(conll2003.iob_sents()[3])

[('West', 'NNP', 'I-NP'), ('Indian', 'NNP', 'I-NP'), ('all-rounder', 'NN', 'I-NP'), ('Phil', 'NNP', 'I-NP'), ('Simmons', 'NNP', 'I-NP'), ('took', 'VBD', 'I-VP'), ('four', 'CD', 'I-NP'), ('for', 'IN', 'I-PP'), ('38', 'CD', 'I-NP'), ('on', 'IN', 'I-PP'), ('Friday', 'NNP', 'I-NP'), ('as', 'IN', 'I-PP'), ('Leicestershire', 'NNP', 'I-NP'), ('beat', 'VBD', 'I-VP'), ('Somerset', 'NNP', 'I-NP'), ('by', 'IN', 'I-PP'), ('an', 'DT', 'I-NP'), ('innings', 'NN', 'I-NP'), ('and', 'CC', 'O'), ('39', 'CD', 'I-NP'), ('runs', 'NNS', 'I-NP'), ('in', 'IN', 'I-PP'), ('two', 'CD', 'I-NP'), ('days', 'NNS', 'I-NP'), ('to', 'TO', 'I-VP'), ('take', 'VB', 'I-VP'), ('over', 'IN', 'I-PP'), ('at', 'IN', 'B-PP'), ('the', 'DT', 'I-NP'), ('head', 'NN', 'I-NP'), ('of', 'IN', 'I-PP'), ('the', 'DT', 'I-NP'), ('county', 'NN', 'I-NP'), ('championship', 'NN', 'I-NP'), ('.', '.', 'O')]


Man beachte aber, dass das Beginn-Tag (`B-XXX`) erst dann verwendet wird, wenn es nötig ist, d.h. wenn zwei NEs vom gleichen Typ adjazent sind! <span style="color:red">(Frage am Rande: Macht das die Sache schwieriger oder einfacher?)</span>

Wir können die IOB-Annoation in das gewohnte Format ([IOB2](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging))) umwandeln, um die Ergebnisse besser vergleichen zu können:

In [3]:
conll2003_iob2 = [nltk.chunk.tree2conlltags(
    nltk.chunk.conlltags2tree(sent)) for sent in conll2003.iob_sents()]
print(conll2003_iob2[3])

[('West', 'NNP', 'B-NP'), ('Indian', 'NNP', 'I-NP'), ('all-rounder', 'NN', 'I-NP'), ('Phil', 'NNP', 'I-NP'), ('Simmons', 'NNP', 'I-NP'), ('took', 'VBD', 'B-VP'), ('four', 'CD', 'B-NP'), ('for', 'IN', 'B-PP'), ('38', 'CD', 'B-NP'), ('on', 'IN', 'B-PP'), ('Friday', 'NNP', 'B-NP'), ('as', 'IN', 'B-PP'), ('Leicestershire', 'NNP', 'B-NP'), ('beat', 'VBD', 'B-VP'), ('Somerset', 'NNP', 'B-NP'), ('by', 'IN', 'B-PP'), ('an', 'DT', 'B-NP'), ('innings', 'NN', 'I-NP'), ('and', 'CC', 'O'), ('39', 'CD', 'B-NP'), ('runs', 'NNS', 'I-NP'), ('in', 'IN', 'B-PP'), ('two', 'CD', 'B-NP'), ('days', 'NNS', 'I-NP'), ('to', 'TO', 'B-VP'), ('take', 'VB', 'I-VP'), ('over', 'IN', 'B-PP'), ('at', 'IN', 'B-PP'), ('the', 'DT', 'B-NP'), ('head', 'NN', 'I-NP'), ('of', 'IN', 'B-PP'), ('the', 'DT', 'B-NP'), ('county', 'NN', 'I-NP'), ('championship', 'NN', 'I-NP'), ('.', '.', 'O')]


### Auslesen des NE-Annotationslayers

Dummerweise wird der `ne`-Annotationslayer einfach ignoriert. `ConllCorpusReader` ist nicht dafür gedacht, vier oder mehr Spalten auszugeben :-( 

Als Workaround kann man die Klasse `ConllCorpusReader` in der folgenden Weise erweitern:

In [4]:
from nltk.util import LazyConcatenation, LazyMap


class extendedConllCorpusReader(ConllCorpusReader):
    """
    `ConllCorpusReader` is a corpus reader for CoNLL-style files.
    This extension allows one to access any column via the extra 
    parameter `column`.
    """

    def iob_words(self, fileids=None, tagset=None, column='chunk'):
        """
        :return: a list of word/tag/IOB tuples
        :rtype: list(tuple)
        :param fileids: the list of fileids that make up this corpus
        :type fileids: None or str or list
        """
        self._require(self.WORDS, self.POS, self.CHUNK)

        def get_iob_words(grid):
            return self._get_iob_words(grid, tagset, column)
        return LazyConcatenation(LazyMap(get_iob_words, self._grids(fileids)))

    def iob_sents(self, fileids=None, tagset=None, column='chunk'):
        """
        :return: a list of lists of word/tag/IOB tuples
        :rtype: list(list)
        :param fileids: the list of fileids that make up this corpus
        :type fileids: None or str or list
        """
        self._require(self.WORDS, self.POS, self.CHUNK)

        def get_iob_words(grid):
            return self._get_iob_words(grid, tagset, column)
        return LazyMap(get_iob_words, self._grids(fileids))

    def _get_iob_words(self, grid, tagset=None, column='chunk'):
        pos_tags = self._get_column(grid, self._colmap['pos'])
        if tagset and tagset != self._tagset:
            pos_tags = [map_tag(self._tagset, tagset, t) for t in pos_tags]
        return list(zip(self._get_column(grid, self._colmap['words']), pos_tags,
                        self._get_column(grid, self._colmap[column])))


conll2003 = extendedConllCorpusReader(
    'conll2003/', '.*', columntypes=('words', 'pos', 'chunk', 'ne'))

Nun kann man den `ne`-Layer mittels des Parameters `column` in dem schon bekannten IOB(1)-Schema ausgeben:  

In [5]:
print(conll2003.iob_sents('eng.train', column='ne'))

[[], [('EU', 'NNP', 'I-ORG'), ('rejects', 'VBZ', 'O'), ('German', 'JJ', 'I-MISC'), ('call', 'NN', 'O'), ('to', 'TO', 'O'), ('boycott', 'VB', 'O'), ('British', 'JJ', 'I-MISC'), ('lamb', 'NN', 'O'), ('.', '.', 'O')], ...]


Der CoNLL2003 Shared Task unterscheidet die vier NE-Typen:
- ORG(ANIZATION) 
- PER(SON)
- LOC(ATION)
- MISC(ELLANEOUS)

### Vereinheitlichung

Für die weitere Nutzung werden die Daten in drei Schritten zusammengeführt und vereinheitlicht:
1. Umwandelung der Annotation in das IOB2-Format
2. Unifikation der IOB-Annotationslayer für Chunks und NEs $\to$ Quadrupel als Worttoken
3. Leere Sätze werden herausgefiltert.

In [6]:
def iob1_to_iob2(chunked_corpus):
    """
    Convert chunked corpus from IOB1 to IOB2 annotation format.
    """
    return [nltk.chunk.tree2conlltags(nltk.chunk.conlltags2tree(sent)) for sent in chunked_corpus]


def merge_iob(c1, c2):
    """
    Merge chunk and NE annotation layers in c1 and c2, which 
    are outputs of iob_sents applied to the same corpus.
    """
    from tqdm import tqdm
    out = []
    for i in tqdm(range(len(c1))):
        out.append([(word, pos, chunk, ne)
                   for (word, pos, chunk), (word, pos, ne) in zip(c1[i], c2[i])])
    return out


conll2003_ioball_train = list(filter(None, merge_iob(iob1_to_iob2(conll2003.iob_sents('eng.train', column='chunk')),
                                                     iob1_to_iob2(conll2003.iob_sents('eng.train', column='ne')))))
conll2003_ioball_testa = list(filter(None, merge_iob(iob1_to_iob2(conll2003.iob_sents('eng.testa', column='chunk')),
                                                     iob1_to_iob2(conll2003.iob_sents('eng.testa', column='ne')))))
conll2003_ioball_testb = list(filter(None, merge_iob(iob1_to_iob2(conll2003.iob_sents('eng.testb', column='chunk')),
                                                     iob1_to_iob2(conll2003.iob_sents('eng.testb', column='ne')))))

100%|███████████████████████████████████| 3684/3684 [00:00<00:00, 393155.97it/s]


Das Ergebnis sieht so aus:

In [7]:
conll2003_ioball_train

[[('EU', 'NNP', 'B-NP', 'B-ORG'),
  ('rejects', 'VBZ', 'B-VP', 'O'),
  ('German', 'JJ', 'B-NP', 'B-MISC'),
  ('call', 'NN', 'I-NP', 'O'),
  ('to', 'TO', 'B-VP', 'O'),
  ('boycott', 'VB', 'I-VP', 'O'),
  ('British', 'JJ', 'B-NP', 'B-MISC'),
  ('lamb', 'NN', 'I-NP', 'O'),
  ('.', '.', 'O', 'O')],
 [('Peter', 'NNP', 'B-NP', 'B-PER'), ('Blackburn', 'NNP', 'I-NP', 'I-PER')],
 [('BRUSSELS', 'NNP', 'B-NP', 'B-LOC'), ('1996-08-22', 'CD', 'I-NP', 'O')],
 [('The', 'DT', 'B-NP', 'O'),
  ('European', 'NNP', 'I-NP', 'B-ORG'),
  ('Commission', 'NNP', 'I-NP', 'I-ORG'),
  ('said', 'VBD', 'B-VP', 'O'),
  ('on', 'IN', 'B-PP', 'O'),
  ('Thursday', 'NNP', 'B-NP', 'O'),
  ('it', 'PRP', 'B-NP', 'O'),
  ('disagreed', 'VBD', 'B-VP', 'O'),
  ('with', 'IN', 'B-PP', 'O'),
  ('German', 'JJ', 'B-NP', 'B-MISC'),
  ('advice', 'NN', 'I-NP', 'O'),
  ('to', 'TO', 'B-PP', 'O'),
  ('consumers', 'NNS', 'B-NP', 'O'),
  ('to', 'TO', 'B-VP', 'O'),
  ('shun', 'VB', 'I-VP', 'O'),
  ('British', 'JJ', 'B-NP', 'B-MISC'),
  ('lamb

Damit wir die üblichen NLTK-Interfaces nutzen können, werden wir uns auf Trippel bestehend aus Wortformen, Chunk-Tags und NE-Tags beschränken. Diese müssen als Tree-Objekte vorliegen, was mit dem folgenden Code erreicht wird:

In [8]:
conll2003_tree_testa = [nltk.chunk.conlltags2tree(sent)
                        for sent
                        in [[(word, (pos, chunk), ne)
                             for word, pos, chunk, ne
                             in sent2] for sent2 in conll2003_ioball_testa]]
conll2003_tree_testb = [nltk.chunk.conlltags2tree(sent)
                        for sent
                        in [[(word, (pos, chunk), ne)
                             for word, pos, chunk, ne
                             in sent2] for sent2 in conll2003_ioball_testb]]
conll2003_tree_train = [nltk.chunk.conlltags2tree(sent)
                        for sent
                        in [[(word, (pos, chunk), ne)
                             for word, pos, chunk, ne
                             in sent2] for sent2 in conll2003_ioball_train]]
conll2003_tree_train

[Tree('S', [Tree('ORG', [('EU', ('NNP', 'B-NP'))]), ('rejects', ('VBZ', 'B-VP')), Tree('MISC', [('German', ('JJ', 'B-NP'))]), ('call', ('NN', 'I-NP')), ('to', ('TO', 'B-VP')), ('boycott', ('VB', 'I-VP')), Tree('MISC', [('British', ('JJ', 'B-NP'))]), ('lamb', ('NN', 'I-NP')), ('.', ('.', 'O'))]),
 Tree('S', [Tree('PER', [('Peter', ('NNP', 'B-NP')), ('Blackburn', ('NNP', 'I-NP'))])]),
 Tree('S', [Tree('LOC', [('BRUSSELS', ('NNP', 'B-NP'))]), ('1996-08-22', ('CD', 'I-NP'))]),
 Tree('S', [('The', ('DT', 'B-NP')), Tree('ORG', [('European', ('NNP', 'I-NP')), ('Commission', ('NNP', 'I-NP'))]), ('said', ('VBD', 'B-VP')), ('on', ('IN', 'B-PP')), ('Thursday', ('NNP', 'B-NP')), ('it', ('PRP', 'B-NP')), ('disagreed', ('VBD', 'B-VP')), ('with', ('IN', 'B-PP')), Tree('MISC', [('German', ('JJ', 'B-NP'))]), ('advice', ('NN', 'I-NP')), ('to', ('TO', 'B-PP')), ('consumers', ('NNS', 'B-NP')), ('to', ('TO', 'B-VP')), ('shun', ('VB', 'I-VP')), Tree('MISC', [('British', ('JJ', 'B-NP'))]), ('lamb', ('NN', 'I

Für die Tagger-Schnittstelle benötigen man dagegen Paare als Datenformat:

In [9]:
conll2003_pair_testa = [[((word, (pos, chunk)), ne) for word, pos, chunk, ne in sent]
                        for sent in conll2003_ioball_testa]
conll2003_pair_testb = [[((word, (pos, chunk)), ne) for word, pos, chunk, ne in sent]
                        for sent in conll2003_ioball_testb]
conll2003_pair_train = [[((word, (pos, chunk)), ne) for word, pos, chunk, ne in sent]
                        for sent in conll2003_ioball_train]

In [10]:
conll2003_pair_train 

[[(('EU', ('NNP', 'B-NP')), 'B-ORG'),
  (('rejects', ('VBZ', 'B-VP')), 'O'),
  (('German', ('JJ', 'B-NP')), 'B-MISC'),
  (('call', ('NN', 'I-NP')), 'O'),
  (('to', ('TO', 'B-VP')), 'O'),
  (('boycott', ('VB', 'I-VP')), 'O'),
  (('British', ('JJ', 'B-NP')), 'B-MISC'),
  (('lamb', ('NN', 'I-NP')), 'O'),
  (('.', ('.', 'O')), 'O')],
 [(('Peter', ('NNP', 'B-NP')), 'B-PER'),
  (('Blackburn', ('NNP', 'I-NP')), 'I-PER')],
 [(('BRUSSELS', ('NNP', 'B-NP')), 'B-LOC'),
  (('1996-08-22', ('CD', 'I-NP')), 'O')],
 [(('The', ('DT', 'B-NP')), 'O'),
  (('European', ('NNP', 'I-NP')), 'B-ORG'),
  (('Commission', ('NNP', 'I-NP')), 'I-ORG'),
  (('said', ('VBD', 'B-VP')), 'O'),
  (('on', ('IN', 'B-PP')), 'O'),
  (('Thursday', ('NNP', 'B-NP')), 'O'),
  (('it', ('PRP', 'B-NP')), 'O'),
  (('disagreed', ('VBD', 'B-VP')), 'O'),
  (('with', ('IN', 'B-PP')), 'O'),
  (('German', ('JJ', 'B-NP')), 'B-MISC'),
  (('advice', ('NN', 'I-NP')), 'O'),
  (('to', ('TO', 'B-PP')), 'O'),
  (('consumers', ('NNS', 'B-NP')), 'O'),

### Übersicht der Formate

Nach dem Einlesen der CoNLL-2003-Daten stehen die folgenden Formate zu Verfügung:

| **Funktion/Variable**                          | **Tokenrepräsentation**                  |
|:-----------------------------------------------|:-----------------------------------------|
| `conll2003.iob_words('eng.train',column='ne')` | `('EU', 'NNP', 'I-ORG')`                 |
| `conll2003_ioball_train`                       | `('EU', 'NNP', 'B-NP', 'B-ORG')`         |
| `conll2003_pair_train`                         | `(('EU', ('NNP', 'B-NP')), 'B-ORG')`     |
| `conll2003_tree_train`                         | `Tree('ORG', [('EU', ('NNP', 'B-NP'))])` |



## CoNLL-2003 Shared Task: Statistik der Trainingsdaten

Die Einteilung in Trainings-, Entwicklungs- und Testdaten ist beim CoNLL-2003 Shared Task vorgegeben. Um die Annotation etwas besser kennenzulernen, werfen wir einen Blick auf die Trainingsdaten.

In [11]:
print([token for token in conll2003.iob_words('eng.train', column='ne')[:50]])

[('EU', 'NNP', 'I-ORG'), ('rejects', 'VBZ', 'O'), ('German', 'JJ', 'I-MISC'), ('call', 'NN', 'O'), ('to', 'TO', 'O'), ('boycott', 'VB', 'O'), ('British', 'JJ', 'I-MISC'), ('lamb', 'NN', 'O'), ('.', '.', 'O'), ('Peter', 'NNP', 'I-PER'), ('Blackburn', 'NNP', 'I-PER'), ('BRUSSELS', 'NNP', 'I-LOC'), ('1996-08-22', 'CD', 'O'), ('The', 'DT', 'O'), ('European', 'NNP', 'I-ORG'), ('Commission', 'NNP', 'I-ORG'), ('said', 'VBD', 'O'), ('on', 'IN', 'O'), ('Thursday', 'NNP', 'O'), ('it', 'PRP', 'O'), ('disagreed', 'VBD', 'O'), ('with', 'IN', 'O'), ('German', 'JJ', 'I-MISC'), ('advice', 'NN', 'O'), ('to', 'TO', 'O'), ('consumers', 'NNS', 'O'), ('to', 'TO', 'O'), ('shun', 'VB', 'O'), ('British', 'JJ', 'I-MISC'), ('lamb', 'NN', 'O'), ('until', 'IN', 'O'), ('scientists', 'NNS', 'O'), ('determine', 'VBP', 'O'), ('whether', 'IN', 'O'), ('mad', 'JJ', 'O'), ('cow', 'NN', 'O'), ('disease', 'NN', 'O'), ('can', 'MD', 'O'), ('be', 'VB', 'O'), ('transmitted', 'VBN', 'O'), ('to', 'TO', 'O'), ('sheep', 'NN', 'O')

Es sollte nicht überraschen, dass `O`, also keine Named Entity, das häufigste NE-Tag ist:

In [12]:
from nltk.probability import FreqDist

corpstat = FreqDist(conll2003.iob_words('eng.train', column='ne'))
netagstat = FreqDist(
    [netag for word, pos, netag in conll2003.iob_words('eng.train', column='ne')])

print("Häufigste Token: ", corpstat.most_common(10), "\n")
print("Häufigkeit der NE-Tags: ", netagstat.most_common(10))

Häufigste Token:  [(('.', '.', 'O'), 7362), ((',', ',', 'O'), 7275), (('the', 'DT', 'O'), 7221), (('of', 'IN', 'O'), 3617), (('to', 'TO', 'O'), 3382), (('in', 'IN', 'O'), 3370), (('a', 'DT', 'O'), 2993), (('(', '(', 'O'), 2846), ((')', ')', 'O'), 2846), (('and', 'CC', 'O'), 2789)] 

Häufigkeit der NE-Tags:  [('O', 169578), ('I-PER', 11128), ('I-ORG', 10001), ('I-LOC', 8286), ('I-MISC', 4556), ('B-MISC', 37), ('B-ORG', 24), ('B-LOC', 11)]


Wie beim POS-Tagging ist die Ambiguität der Wort**token** weitaus ausgeprägter als die Ambiguität der Wort**formen**:

In [13]:
# Create a dictionary to store the list of NE tags per word form
netagdict = {}

for word, pos, netag in corpstat.keys():
    if word in netagdict:
        if netag not in netagdict[word]:
            netagdict[word] = netagdict[word] + [netag]
    else:
        netagdict[word] = [netag]

netagdict

{'EU': ['I-ORG'],
 'rejects': ['O'],
 'German': ['I-MISC', 'I-ORG'],
 'call': ['O'],
 'to': ['O'],
 'boycott': ['O'],
 'British': ['I-MISC', 'I-ORG', 'I-LOC'],
 'lamb': ['O'],
 '.': ['O', 'I-MISC', 'I-LOC', 'I-ORG'],
 'Peter': ['I-PER'],
 'Blackburn': ['I-PER', 'I-ORG'],
 'BRUSSELS': ['I-LOC', 'I-MISC'],
 '1996-08-22': ['O'],
 'The': ['O', 'I-LOC', 'I-ORG', 'I-MISC'],
 'European': ['I-ORG', 'I-MISC'],
 'Commission': ['I-ORG', 'I-MISC'],
 'said': ['O'],
 'on': ['O', 'I-MISC'],
 'Thursday': ['O'],
 'it': ['O'],
 'disagreed': ['O'],
 'with': ['O'],
 'advice': ['O'],
 'consumers': ['O'],
 'shun': ['O'],
 'until': ['O'],
 'scientists': ['O'],
 'determine': ['O'],
 'whether': ['O'],
 'mad': ['O'],
 'cow': ['O'],
 'disease': ['O'],
 'can': ['O'],
 'be': ['O'],
 'transmitted': ['O'],
 'sheep': ['O'],
 'Germany': ['I-LOC', 'I-ORG'],
 "'s": ['O', 'I-ORG', 'I-MISC', 'I-LOC'],
 'representative': ['O'],
 'the': ['O', 'I-ORG', 'I-PER', 'I-MISC', 'I-LOC'],
 'Union': ['I-ORG', 'I-LOC', 'O'],
 'veterin

In [14]:
# Average number of NE tags per word form
from statistics import mean
mean([len(netaglist) for netaglist in netagdict.values()])

1.0486390382254582

In [15]:
# Average number of potential NE tags per word token
mean([len(netagdict[word])
     for word, pos, netag in conll2003.iob_words('eng.train', column='ne')])

1.7827778077899628

## NER als überwacht gelerntes N-Gramm-Tagging

Die n-Gramm-Ansätze, die wir in den letzten Sitzungen kennengelernt haben, betrachten immer Kontexte fester Größe und Richtung, wobei durch Backoff-Strategien eine gewisse Flexiblität erreicht werden kann.

![](https://www.nltk.org/images/tag-context.png)

Dies ist jedoch nicht zufriedenstellend, da man im Grunde immer nur ein Feature betrachtet. Man möchte aber gerne, und zwar ohne komplizierte Datenmassage, gleichzeitig **unterschiedliche Features** (Wortform, POS-Tag, Chunk-Tag, ...) betrachten können und auch in der Lage sein, **unterschiedliche Strategien** zu verfolgen. 

Beispielsweise möchte man bei PER vielleicht das erste Token eines Chunks berücksichtigen und in einer Namensliste nachschlagen (*Michael London*). Dagegen ist bei LOC und ORG das letzte Token (der "Kopf") relevant (*Stanford University*).   

Um das zu erreichen, könnte man einen regelbasierten Ansatz verwenden. Im Folgenden wollen wir aber bei den daten-getriebenen Verfahren bleiben und den NE-Chunker als **überwacht gelernten Klassifikator** implementieren. Wir beschränken uns hier auf die Naiv-Bayes-Klassifizierung, die gerne zur Berechnung der Baseline eingesetzt wird und mit der trotz der simplen Grundidee bereits gute Ergebnisse erreicht werden können.

## Feature-basierte Klassifikation: Naive-Bayes-Klassifikator

### Feature-basierte Klassifikation

Die Idee der feature-basierten Klassifikation ist recht simpel: Ein **Feature** stellt eine konkrete Konfiguration des (vertikalen oder horizontalen) Kontexts eines Tokens dar, z.B. dass links daneben ein Satzzeichen steht, oder dass sich das Token in einem NP-Chunk. 

Während des **Lernvorgangs** wird ermittelt, wie häufig ein Token mit einem Label und diesem Feature (= einer bestimmten Konfiguration) auftritt und das Feature wird entsprechend für jedes Label gewichtet. Beim **Klassifizieren** wird dann dasjenige Label für ein Target ausgewählt, das entsprechend der Features, die beim Target gefunden werden, das höchste Gewicht hat.  

Schematisch wird das im NLTK-Buch so dargestellt:

<img src="https://www.nltk.org/images/supervised-classification.png" width="70%"/>

Das heißt, letztlich arbeitet der Klassifizierer mit einer großen Menge von Features, die zuvor in Form eines Feature-Vektors aus den Sprachdaten "extrahiert" werden müssen. Die Features (bzw. deren Templates) muss man sich selber vorher ausdenken, d.h. man muss überlegen, wo die für diese Klasssifkation wesentlichen Informationen in den Sprachdaten stecken.

Für die **Gewichtung der Feature-Label-Kombinationen** (siehe "machine learning algorithm" und "classifier model" im Schema) gibt es eine Reihe von Verfahren. Wir werden uns im Weiteren mit einem recht simplen Verfahren beschäftigen, mit dem sich aber schon gute Ergebnisse erreichen lassen: Das Naive-Bayes-Verfahren.  

### Hintergrund: Satz von Bayes

Nehmen wir an, dass $a$ und $b$ zwei Eigenschaften/Merkmale/Features von Objekten/Ereignissen/Messungen einer Grundgesamtheit sind, die diese haben können oder auch nicht. Wir können dann aus einer Reihe von Beobachtungen von $a$ und $b$ in einer Stichprobe mehr oder weniger zuverlässig auf deren "marginale" Wahrscheinlichkeit $p(a)$ und $p(b)$ und deren gemeinsame Wahrscheinlichkeit $p(a,b)$ schließen. 

Wollen wir dann $a$ aus der Beobachtung von $b$ vorhersagen, dann betrachten wir die **bedingte Wahrscheinlichkeit** $p(a\vert b)$, d.h. die Frage ist hier: Wie wahrscheinlich ist $a$, wenn wir nur die Objekte betrachten, in denen $b$ vorliegt? Wir betrachten also die die gemeinsame Wahrscheinlichkeit von $a$ und $b$ im Verhältnis zur Wahrscheinlichkeit von $b$:

$$p(a\vert b) = \frac{p(a,b)}{p(b)}$$

Dies gilt genauso für $p(b\vert a)$:

$$p(b\vert a) = \frac{p(a,b)}{p(a)}$$

Umgekehrt gilt trivialerweise auch:

$$p(a,b) = p(a\vert b) ~ p(b)$$

und 

$$p(a,b) = p(b\vert a) ~ p(a)$$

Wir können also $p(a\vert b)$ auch ohne $p(a,b)$ angeben und dabei die bedingte Wahrscheinlichkeit quasi umdrehen:

$$p(a|b) = \frac{p(b\vert a) ~ p(a)}{p(b)}$$

Genau dies (und nicht mehr) beinhaltet der [Satz von Bayes](https://en.wikipedia.org/wiki/Bayes%27_theorem).

###  Hintergrund: Naive-Bayes-Klassifikator

Ein Klassifikator ist eine Funktion, die Objekten/Ereignissen/Messungen abhängig von deren Features/Eigenschaften ein Klassenlabel zuweist. Diese Zuweisung kann je nach Ansatz unterschiedlich begründet sein. 

Beim [Naive-Bayes-Klassifikator](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) dienen Wahrscheinlichkeiten aus Beispielklassifikationen als Grundlage für die Zuweisung eines Klassenlabels. Da man die Klassenlabel vorhersagen möchte, betrachtet man die bedingten Wahrscheinlichkeiten aller Klassenlabel $C_K$ mit $1 \leq k \leq K$ abhängig von bestimmten beobachteten Features $\mathbf{x} = x_1, \dots, x_n$:  

$$p(C_k \mid x_1, \dots, x_n)$$

Dank des Satzes von Bayes können wir dies so umformulieren:

\begin{align}
p(C_k \mid x_1, \dots, x_n) & = \frac{p(C_k) \ p(x_1, \dots, x_n \mid C_k)}{p(x_1, \dots, x_n)} \\
& = \frac{p(x_1, \dots, x_n, C_k)}{p(x_1, \dots, x_n)} \\
& \varpropto p(x_1, \dots, x_n, C_k)
\end{align}

D.h. bei den Klassenlabeln handelt es sich eigentlich auch um Features wie oben.

Das Problem an dieser Formel ist, dass die Menge (oder der Vektor) der Features $\mathbf{x}$ sehr umfangreich werden kann. Man kann sich $\mathbf{x}$ am besten als binären Vektor vorstellen, in dem jede Position einer Feature entspricht und $0$ das Fehlen und $1$ das Vorhandensein der Feature signalisiert. Wir sprechen also von $K*2^{|\mathbf{x}|}$ bedingten Wahrscheinlichkeiten, die berechnet werden müssten. Bei tausenden von Features, kommt da einiges zusammen.

Die Lösung besteht darin, so zu tun, als ob die Features in $\mathbf{x}$ voneinander paarweise unabhängig sind. Diese Annahme ist reichlich "naiv" (daher der Name), aber nützlich, denn dann können wir die bedingte Wahrscheinlichkeit folgendermaßen approximieren:

\begin{align}
p(C_k \mid x_1, \dots, x_n) & \varpropto p(C_k) \ p(x_1, \dots, x_n \mid C_k)\\
& = p(C_k) ~ \displaystyle\prod_{i=1}^n ~ p(x_i\vert C_k)\\
\end{align}

Mit diesem Trick müssen nur noch $K + K * 2|\mathbf{x}|$ Wahrscheinlichkeiten berechnet werden.

Schließlich bleibt noch zu klären, welches Klassenlabel bei bestimmten Features die höchste Wahrscheinlichkeit hat:

$$ \hat{C} = \underset{k \in \{1, \dots, K\}}{\operatorname{argmax}} \ p(C_k) \displaystyle\prod_{i=1}^n p(x_i \vert C_k)$$

## Feature-Extraktion

Feature-basierte Klassifizierer nutzen 10-tausende von Features, die automatisch mittels **Feature-Templates** aus Lerndaten extrahiert und gewichtet werden. 

In NLTK werden Feature-Templates mittels Funktionen wie `ne_features` manuell spezifiziert, wobei diese Funktionen immer drei Argumente benötigen:

- `sentence`: der Satz als Liste von Token, hier bestehend aus Paare der Form `(word, (pos, chunk))`.
- `i`: der (Positions-)Index des Targets im Satz
- `history`: eine Liste der schon zugewiesenen NE-Tags

Die Features werden für jedes Token als Dictionary ausgegeben, d.h. jedes Feature entspricht einem Eintrag im Dictionary.

In [16]:
def ne_features(sentence, i, history):
    word, (pos, chunk) = sentence[i]
    if i == 0:
        # Padding beim ersten Token
        prevword, prevpos, prevchunk = "<START>", "<START>", "<START>"
    else:
        prevword, (prevpos, prevchunk) = sentence[i-1]
    if i == len(sentence)-1:
        nextword, nextpos = "<END>", "<END>"  # Padding beim letzten Token
    else:
        nextword, (nextpos, nextchunk) = sentence[i+1]
    # Feature-Templates
    return {"pos": pos,
            "word": word,
            "prevpos": prevpos,
            "nextpos": nextpos,
            "prevpos+pos": "%s+%s" % (prevpos, pos),
            "pos+nextpos": "%s+%s" % (pos, nextpos),
            }

Lassen wir ein Beispiel sprechen:

In [17]:
# NE-Tags müssen vorher aus dem Trainings-Set entfernt werden.
sentence = list(zip(*conll2003_pair_train[10]))[0]
print(sentence)
ne_features(sentence, 1, [])

(('Spanish', ('NNP', 'B-NP')), ('Farm', ('NNP', 'I-NP')), ('Minister', ('NNP', 'I-NP')), ('Loyola', ('NNP', 'I-NP')), ('de', ('NNP', 'I-NP')), ('Palacio', ('NNP', 'I-NP')), ('had', ('VBD', 'B-VP')), ('earlier', ('RBR', 'I-VP')), ('accused', ('VBN', 'I-VP')), ('Fischler', ('NNP', 'B-NP')), ('at', ('IN', 'B-PP')), ('an', ('DT', 'B-NP')), ('EU', ('JJ', 'I-NP')), ('farm', ('NN', 'I-NP')), ('ministers', ('NNS', 'I-NP')), ("'", ('POS', 'B-NP')), ('meeting', ('NN', 'I-NP')), ('of', ('IN', 'B-PP')), ('causing', ('VBG', 'B-VP')), ('unjustified', ('JJ', 'B-ADJP')), ('alarm', ('NN', 'B-NP')), ('through', ('IN', 'B-PP')), ('"', ('"', 'O')), ('dangerous', ('JJ', 'B-NP')), ('generalisation', ('NN', 'I-NP')), ('.', ('.', 'O')), ('"', ('"', 'O')))


{'pos': 'NNP',
 'word': 'Farm',
 'prevpos': 'NNP',
 'nextpos': 'NNP',
 'prevpos+pos': 'NNP+NNP',
 'pos+nextpos': 'NNP+NNP'}

## Klassifikation

Als nächstes wird die Klasse `NamedEntityChunker` definiert, wobei der eingebettete Tagger eine Instanz der Klasse `ClassifierBasedTagger` ist. 

In [18]:
from collections.abc import Iterable
from nltk.tag import ClassifierBasedTagger, DefaultTagger
from nltk.chunk import ChunkParserI
from nltk.classify import NaiveBayesClassifier


class NamedEntityChunker(ChunkParserI):
    """
    Class for classifier-based chunkers for named entities.
    The classifier can be chosen with parameter `classifier_builder` during initalization. The
    default is `NaiveBayesClassifier.train`.
    Features are included via the function `ne_features`.
    The `backoff` tagger is called whenever probability falls below `cutoff_prob`. 
    """

    def __init__(self, train_sents,
                 classifier_builder=NaiveBayesClassifier.train,
                 feature_generator=ne_features,
                 backoff=DefaultTagger('O'),
                 cutoff_prob=0.3,
                 **kwargs):
        assert isinstance(train_sents, Iterable)

        self.feature_detector = ne_features
        self.tagger = ClassifierBasedTagger(
            train=train_sents,
            feature_detector=feature_generator,
            classifier_builder=classifier_builder,
            backoff=backoff,
            cutoff_prob=cutoff_prob,
            **kwargs)

    def parse(self, tagged_sent):
        ne_tagged_sent = self.tagger.tag(tagged_sent)

        # Convert tagger pairs to chunker triples
        ne_tuples = [(w, (t, c), ne) for ((w, (t, c)), ne) in ne_tagged_sent]

        # Transform the list of iob-triples to nltk.Tree format
        return nltk.conlltags2tree(ne_tuples)

Bei der Initialisierung eines NE-Chunkers können wir optional einen Klassifizierer und den Feature-Generator angeben. Der Default ist `NaiveBayesClassifier.train` und `ne_features`:

In [19]:
from nltk.classify import NaiveBayesClassifier

ne_chunker = NamedEntityChunker(conll2003_pair_train)

Nach der Trainingsphase kann der Tagger verwendet werden:

In [20]:
# The input of the tagger must be untagged.
ne_chunker.tagger.tag(list(zip(*conll2003_pair_testa[2]))[0])

[(('West', ('NNP', 'B-NP')), 'B-ORG'),
 (('Indian', ('NNP', 'I-NP')), 'I-MISC'),
 (('all-rounder', ('NN', 'I-NP')), 'O'),
 (('Phil', ('NNP', 'I-NP')), 'B-PER'),
 (('Simmons', ('NNP', 'I-NP')), 'I-PER'),
 (('took', ('VBD', 'B-VP')), 'O'),
 (('four', ('CD', 'B-NP')), 'O'),
 (('for', ('IN', 'B-PP')), 'O'),
 (('38', ('CD', 'B-NP')), 'O'),
 (('on', ('IN', 'B-PP')), 'O'),
 (('Friday', ('NNP', 'B-NP')), 'B-LOC'),
 (('as', ('IN', 'B-PP')), 'O'),
 (('Leicestershire', ('NNP', 'B-NP')), 'B-ORG'),
 (('beat', ('VBD', 'B-VP')), 'O'),
 (('Somerset', ('NNP', 'B-NP')), 'B-ORG'),
 (('by', ('IN', 'B-PP')), 'O'),
 (('an', ('DT', 'B-NP')), 'O'),
 (('innings', ('NN', 'I-NP')), 'O'),
 (('and', ('CC', 'O')), 'O'),
 (('39', ('CD', 'B-NP')), 'O'),
 (('runs', ('NNS', 'I-NP')), 'O'),
 (('in', ('IN', 'B-PP')), 'O'),
 (('two', ('CD', 'B-NP')), 'O'),
 (('days', ('NNS', 'I-NP')), 'O'),
 (('to', ('TO', 'B-VP')), 'O'),
 (('take', ('VB', 'I-VP')), 'O'),
 (('over', ('IN', 'B-PP')), 'O'),
 (('at', ('IN', 'B-PP')), 'O'),
 

Da wir es hier mit einer Klasse zu tun haben, die von `ChunkParserI` abgeleitet wurde, steht auch wieder die Funktion `accuracy()` zur Verfügung. Zudem können wir die Funktion `accuracy()` auch auf den einbegetteten Tagger anwenden.

Wie gut mag wohl der klassifiziererbasierte NE-Chunker auf den Entwicklungsdaten abschneiden? 

In [21]:
print("Tagger accuracy: ", ne_chunker.tagger.accuracy(conll2003_pair_testa))
print(ne_chunker.accuracy(conll2003_tree_testa))

Tagger accuracy:  0.8899186168762898
ChunkParse score:
    IOB Accuracy:  88.3%%
    Precision:     42.0%%
    Recall:        67.9%%
    F-Measure:     51.9%%


Die Ergebnisse sind erwartungsgemäß mau, aber wir haben ja die Möglichkeiten noch längst nicht ausgereitzt. 

Werfen wir einen Blick auf die Verwechslungsmatrix, um Verbesserungsmöglichkeiten zu finden:

In [22]:
from nltk.metrics import ConfusionMatrix
from nltk import conlltags2tree, tree2conlltags

corpus = conll2003_pair_testa

gold = [netag for sent in corpus for (word, (pos, chunk)), netag in sent]
test = [netag for sent
        in [ne_chunker.tagger.tag(list(zip(*sent))[0]) for sent in corpus]
        for (word, (pos, chunk)), netag in sent]

print(ConfusionMatrix(gold, test).pretty_format(show_percents=True,
      values_in_chart=True, truncate=15, sort_by_count=True))

       |                                         B             I        |
       |             B      B      B      I      -      I      -      I |
       |             -      -      -      -      M      -      M      - |
       |             P      L      O      P      I      O      I      L |
       |             E      O      R      E      S      R      S      O |
       |      O      R      C      G      R      C      G      C      C |
-------+----------------------------------------------------------------+
     O | <76.9%>  0.5%   1.1%   0.4%   0.3%   3.5%   0.4%   0.1%   0.0% |
 B-PER |   0.1%  <2.9%>  0.3%   0.1%   0.0%   0.1%   0.1%   0.0%   0.0% |
 B-LOC |   0.1%   0.2%  <2.7%>  0.3%   0.1%   0.2%   0.0%   0.0%      . |
 B-ORG |   0.1%   0.3%   0.3%  <1.7%>  0.0%   0.1%   0.0%   0.0%      . |
 I-PER |   0.1%   0.0%   0.0%   0.0%  <2.2%>  0.0%   0.2%   0.0%   0.0% |
B-MISC |   0.1%   0.1%   0.1%   0.2%   0.0%  <1.2%>  0.0%   0.0%      . |
 I-ORG |   0.2%   0.0%   0.1%   0.0%  

Aus der Verwechslungsmatrix geht u.a. hervor, dass viele `O`-Token irrtümlich mit NE-Tags versehen wurden. Das spricht dafür, dass weitere Feature-Templates bei der Feature-Extraktion berücksichtigt werden sollten, da die bestehenden Features den Chunker manchmal in die Irre führen. <span style="color:red">(Was könnte man außerdem tun, ohne die Feature-Extraktion zu optimieren?)</span>

## <span style="color:red">Aufgaben I: Mehr Kontext</span>

Zweifelsohne macht die Feature-Extraktion noch zu wenig Gebrauch vom vertikalen und horizontalen Kontext des Target-Tokens und den orthographischen Eigenschaften. 

<span style="color:red">A1:</span> Erweitern Sie die Funktion `ne_features2` im nächsten Code-Block um die folgenden Feature-Templates:
1. `prevchunk+chunk`: das Chunk des Targets und das Chunk des vorangehenden Tokens
2. `prevne`: das NE-Tag des vorangehenden Tokens (falls vorhanden)
3. `capitalized?`: Ist die Wortform des Tokens groß geschrieben?
4. `abbreviation?`: Ist die Wortform des Tokens eine Abkürzung?

Gelingt es Ihnen damit (oder mit einer Auswahl), das F-Maß auf über 60% zu heben?

In [23]:
# Lösung A1 

# Meine Lösung for Aufgabe A1 
def ne_features2(sentence, i, history):
    features = {}

    # prevchunk+chunk 
    if i > 0:
        prev_chunk = sentence[i - 1][1][1]       # chunk des vorherigen Tokens
        curr_chunk = sentence[i][1][1]           # chunk des aktuellen Tokens
        features['prevchunk+chunk'] = f'{prev_chunk}+{curr_chunk}'

    # prevne 
    if i > 0:
        # Sicherstellen, dass sentence[i - 1] die erforderliche Structure und das NE-Tag hat
        if len(sentence[i - 1][1]) > 2 and sentence[i - 1][1][2] != 'O':      
            prev_ne = sentence[i - 1][1][2]     # NE tag des vorherigen Tokens
            features['prevne'] = prev_ne

    # capitalized? 
    word = sentence[i][0]  
    features['capitalized?'] = word[0].isupper()

    # abbreviation? Assuming alle abbr. sind mit uppercase
    features['abbreviation?'] = word.isupper()

    # mit context: wort, POS, and chunk tags
    features['word'] = sentence[i][0]          # Das wort
    features['pos'] = sentence[i][1][0]        # POS tag des aktuellen Tokens
    features['chunk'] = sentence[i][1][1]      # Chunk tag des aktuellen Tokens

    return features

In [24]:
# Test A1 (nicht verändern)

ne_chunker2 = NamedEntityChunker(
    conll2003_pair_train, feature_generator=ne_features2)

print("Tagger accuracy: ", ne_chunker2.tagger.accuracy(conll2003_pair_testa))
print(ne_chunker2.accuracy(conll2003_tree_testa))

Tagger accuracy:  0.9218293680152642
ChunkParse score:
    IOB Accuracy:  92.3%%
    Precision:     54.1%%
    Recall:        71.4%%
    F-Measure:     61.5%%


In [25]:
from nltk.metrics import ConfusionMatrix
from nltk import conlltags2tree, tree2conlltags

corpus = conll2003_pair_testa

gold = [netag for sent in corpus for (word, (pos, chunk)), netag in sent]
test = [netag for sent
        in [ne_chunker2.tagger.tag(list(zip(*sent))[0]) for sent in corpus]
        for (word, (pos, chunk)), netag in sent]

print(ConfusionMatrix(gold, test).pretty_format(show_percents=True,
      values_in_chart=True, truncate=15, sort_by_count=True))

       |                                         B             I        |
       |             B      B      B      I      -      I      -      I |
       |             -      -      -      -      M      -      M      - |
       |             P      L      O      P      I      O      I      L |
       |             E      O      R      E      S      R      S      O |
       |      O      R      C      G      R      C      G      C      C |
-------+----------------------------------------------------------------+
     O | <80.6%>  0.1%   1.3%   0.3%   0.3%   0.2%   0.3%   0.1%      . |
 B-PER |   0.1%  <2.0%>  0.6%   0.1%   0.8%   0.0%   0.0%   0.0%      . |
 B-LOC |   0.1%   0.0%  <3.0%>  0.2%   0.1%   0.1%   0.1%   0.0%   0.0% |
 B-ORG |   0.1%   0.1%   0.4%  <1.4%>  0.3%   0.1%   0.2%   0.0%   0.0% |
 I-PER |   0.1%   0.0%   0.0%   0.0%  <2.4%>  0.0%   0.0%      .      . |
B-MISC |   0.1%   0.0%   0.1%   0.1%   0.1%  <1.3%>  0.0%   0.0%   0.0% |
 I-ORG |   0.2%   0.0%   0.1%   0.0%  

## Abschließende Evaluation mit den Testdaten

Nach Abschluss der Entwicklung des NER-Systems können wir die Evaluation anhand der Testdaten durchführen: 

In [26]:
ne_chunker_final = NamedEntityChunker(
    conll2003_pair_train, feature_generator=ne_features2)
print("Tagger accuracy: ", ne_chunker_final.tagger.accuracy(conll2003_pair_testb))
print(ne_chunker_final.accuracy(conll2003_tree_testb))

Tagger accuracy:  0.9031764832561645
ChunkParse score:
    IOB Accuracy:  90.4%%
    Precision:     45.7%%
    Recall:        61.9%%
    F-Measure:     52.6%%


Die Baseline beim CoNLL-2003 Shared Tast wird aus den NEs der Lerndaten mit eindeutigem NE-Typ gebildet ("entities with unique class in the training data"):

- Precision: 71.91% 
- Recall: 50.90% 
- F1-Masure: 59.61%

Wenn alles gut gegangen ist, sollte das finale NER-System deutlich über der Baseline liegen und sogar vor dem letzten Platz im Wettbewerb landen. 

# Appendix 

## Die Vorverarbeitungspipeline

Im NLTK-Buch stehen die Entitätenextraktion (Entitiy Extraction) und die Relationsextraktion (Relation Extraction) am Ende einer Verarbeitungspipeline, deren Elemente wir bis auf "relation detection" in den vorhergehenden Sitzungen schon kennengelernt haben:

<img src="https://www.nltk.org/images/ie-architecture.png" alt="Drawing" style="width: 500px"/>

Erinnern wir uns mit einem Code-Beispiel:

In [27]:
# nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag

text = "BBDO South in Atlanta, which handles corporate advertising for Georgia-Pacific, will assume additional duties for brands like Angel Soft toilet tissue and Sparkle paper towels, said Ken Haldin, a spokesman for Georgia-Pacific in Atlanta."

# 1) sentence segmentation
text = sent_tokenize(text, language='english')
# 2) tokenization
text = [word_tokenize(sent, language='english') for sent in text]
# 3) part-of-speech tagging
text = [pos_tag(sent, tagset=None, lang='eng') for sent in text]

text

[[('BBDO', 'NNP'),
  ('South', 'NNP'),
  ('in', 'IN'),
  ('Atlanta', 'NNP'),
  (',', ','),
  ('which', 'WDT'),
  ('handles', 'VBZ'),
  ('corporate', 'JJ'),
  ('advertising', 'NN'),
  ('for', 'IN'),
  ('Georgia-Pacific', 'NNP'),
  (',', ','),
  ('will', 'MD'),
  ('assume', 'VB'),
  ('additional', 'JJ'),
  ('duties', 'NNS'),
  ('for', 'IN'),
  ('brands', 'NNS'),
  ('like', 'IN'),
  ('Angel', 'NNP'),
  ('Soft', 'NNP'),
  ('toilet', 'NN'),
  ('tissue', 'NN'),
  ('and', 'CC'),
  ('Sparkle', 'NNP'),
  ('paper', 'NN'),
  ('towels', 'NNS'),
  (',', ','),
  ('said', 'VBD'),
  ('Ken', 'NNP'),
  ('Haldin', 'NNP'),
  (',', ','),
  ('a', 'DT'),
  ('spokesman', 'NN'),
  ('for', 'IN'),
  ('Georgia-Pacific', 'NNP'),
  ('in', 'IN'),
  ('Atlanta', 'NNP'),
  ('.', '.')]]

NLTK enthält keinen fertigen Chunker für das Englische. Wir behelfen uns deshalb mit dem CRF-Chunker aus der letzten Sitzung und trainieren diesen mit den CoNLL-2000-Daten:

In [28]:
from nltk.corpus import conll2000
from nltk.tag import CRFTagger


class CRFChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        train_data = [[(t, c) for w, t, c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        ct = CRFTagger()
        # model.crf.tagger ist nur ein Platzhalter
        ct.train(train_data, 'model.crf.tagger')
        self.tagger = ct

    def parse(self, sentence):
        pos_tags = [pos for (word, pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word, pos), chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)


conlltrain = conll2000.chunked_sents('train.txt')
chunker = CRFChunker(conlltrain)

Hier zur Erinnerung die Performanz dieses Chunkers anhand der CoNLL2000-Testdaten:

In [29]:
conlltest = conll2000.chunked_sents('test.txt')
print(chunker.accuracy(conlltest))

ChunkParse score:
    IOB Accuracy:  92.1%%
    Precision:     87.7%%
    Recall:        89.3%%
    F-Measure:     88.5%%


Jetzt können wir `chunker.parse()` verwenden, um die tokenisierten und mit POS-Tags versehenen Sätze in `text` zu chunken. Die dabei ausgegebenen `Tree`-Objekte konvertieren wir mit `tree2conlltags`:

In [30]:
# 4) entity detection (chunking)
text = [nltk.chunk.tree2conlltags(chunker.parse(sent)) for sent in text]

text

[[('BBDO', 'NNP', 'B-NP'),
  ('South', 'NNP', 'I-NP'),
  ('in', 'IN', 'B-PP'),
  ('Atlanta', 'NNP', 'B-NP'),
  (',', ',', 'O'),
  ('which', 'WDT', 'B-NP'),
  ('handles', 'VBZ', 'B-VP'),
  ('corporate', 'JJ', 'B-NP'),
  ('advertising', 'NN', 'I-NP'),
  ('for', 'IN', 'B-PP'),
  ('Georgia-Pacific', 'NNP', 'B-NP'),
  (',', ',', 'O'),
  ('will', 'MD', 'B-VP'),
  ('assume', 'VB', 'I-VP'),
  ('additional', 'JJ', 'B-NP'),
  ('duties', 'NNS', 'I-NP'),
  ('for', 'IN', 'B-PP'),
  ('brands', 'NNS', 'B-NP'),
  ('like', 'IN', 'B-PP'),
  ('Angel', 'NNP', 'B-NP'),
  ('Soft', 'NNP', 'I-NP'),
  ('toilet', 'NN', 'I-NP'),
  ('tissue', 'NN', 'I-NP'),
  ('and', 'CC', 'O'),
  ('Sparkle', 'NNP', 'B-NP'),
  ('paper', 'NN', 'I-NP'),
  ('towels', 'NNS', 'I-NP'),
  (',', ',', 'O'),
  ('said', 'VBD', 'B-VP'),
  ('Ken', 'NNP', 'B-NP'),
  ('Haldin', 'NNP', 'I-NP'),
  (',', ',', 'O'),
  ('a', 'DT', 'B-NP'),
  ('spokesman', 'NN', 'I-NP'),
  ('for', 'IN', 'B-PP'),
  ('Georgia-Pacific', 'NNP', 'B-NP'),
  ('in', 'IN', 'B

Der letzte Verarbeitungsschritt "entity detection" beinhaltet tatsächlich nur eine **Teilaufgabe der Entitätenextraktion**; es fehlt die Kategorisierung der identifizierten Entitäten. Außerdem möchte man sich bei der Informationsextraktion gerne auf Eigennamen wie *Georgia-Pacific* beschränken und diese grobe ontologische Kategorisierung. Diese Einengung bei der Entitätenextraktion nennt man Named-Entity Recognition (NER).  